In [15]:
import pandas as pd
import nltk
import string
import re
import numpy as np
from tqdm import tqdm
import pandas as pd
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize
import preprocessor as p
from nltk.corpus import stopwords
from tqdm import tqdm

In [66]:
# Function to get All caps count from the tweets
# pass tweets as it is
def FuncAllCapsCount(GivenTweets):
    AllCapsCount=np.zeros(len(GivenTweets))
    for i in tqdm(range(len(GivenTweets))):
        tweet=nltk.word_tokenize(GivenTweets[i])
        for word in tweet:
            if(word!="I" and re.match("^[A-Z]+$",word)):
                AllCapsCount[i]+=1
    return(AllCapsCount)

In [131]:
# Function to get COntinuous Punctuation (???,??!,!!) Count and check if last token was a ? or !
# pass tweets as it is
def FuncPunctuationsCount(GivenTweets):
    ContPunctuationCount=np.zeros(len(GivenTweets))
    LastTokenPunctuation=np.zeros(len(GivenTweets))
    for i in tqdm(range(len(GivenTweets))):
        tweet=nltk.word_tokenize(GivenTweets[i])
        token=0
        while token<len(tweet):
            if(tweet[token]=="?" or tweet[token]=="!"):
                index=token+1
                while(index< len(tweet) and (tweet[index]=="?" or tweet[index]=="!")):
                    index+=1
                if(index-token>1):
                    ContPunctuationCount[i]+=1
                    token=index
            token+=1
        if(tweet[len(tweet)-1]=="?" or tweet[len(tweet)-1]=="!"):
            LastTokenPunctuation[i]+=1
    
    return(ContPunctuationCount,LastTokenPunctuation)

In [141]:
# Fucntion to get HashTag count from the tweets
# pass tweets as it is
def FuncHashtagsCount(GivenTweets):    
    HashTagsCount=np.zeros(len(GivenTweets))
    for i in tqdm(range(len(GivenTweets))):
        tweet=GivenTweets[i].split(" ")
        for word in tweet:
            if(re.match("^#[a-z]",word)):
                HashTagsCount[i]+=1
    return(HashTagsCount)

# Preprocess main function

In [102]:
def preprocess_text(tweet, flag_stemm=False, flag_lemm=False, stop_words_list=None):
    
    ## clean (convert to lowercase and remove punctuations and characters and then strip)
    tweet = re.sub(r'[^\w\s]', '', str(tweet).lower().strip())
    ## Tokenize (convert from string to list) and remove the stop words
    tokenize_tweet = tweet.split()

    if stop_words_list is not None:
        tokenize_tweet = [word for word in tokenize_tweet if word not in stop_words_list]


    ## Stemming (remove -ing, -ly, ...)
    if flag_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        tokenize_tweet = [ps.stem(word) for word in tokenize_tweet]

    ## Lemmatisation (convert the word into root word)
    if flag_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        tokenize_tweet = [lem.lemmatize(word) for word in tokenize_tweet]
#     print(tokenize_tweet)
    ## back to string from list
    tweet = " ".join(tokenize_tweet)
    return tweet

# Preprocess caller function

In [103]:
def preprocess_text_caller(Tweets_List):
    stop_words_list=stopwords.words('english')
    for i in (range(len(Tweets_List))):
        Tweets_List[i]=preprocess_text(Tweets_List[i])

In [142]:
def feature_extraction(Tweets_List):
    #Counting the Capital Words
    df_feature=pd.DataFrame(FuncAllCapsCount(Tweets_List))
    df_feature.rename(columns={0:"all_caps_count"},inplace=True)
#     df_feature["all_caps_count"]=df_feature["all_caps_count"].astype('int32')

    #Counting The Punctuations
    ContPunctuationCount,LastTokenPunctuation=FuncPunctuationsCount(Tweets_List)
    df_feature["Cont_Punc_Count"]=pd.DataFrame(ContPunctuationCount)
    df_feature["Last_Token_Punc"]=pd.DataFrame(LastTokenPunctuation)
    
    #Count the hash tags
    df_feature["HashTag_Count"]=pd.DataFrame(LastTokenPunctuation)
    
    
    df_feature=df_feature.astype('int32')
    return(df_feature)

# Use this for preprocessing

In [105]:
df_training=pd.DataFrame(pd.read_csv("olid-training-v1.0.tsv",sep="\t"))
df_training['tweet_original']=df_training['tweet']
preprocess_text_caller(df_training['tweet'])

<ipython-input-103-2b67d72cee89>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Tweets_List[i]=preprocess_text(Tweets_List[i])


In [143]:
a=feature_extraction(df_training['tweet_original'][0:100])

100%|██████████| 100/100 [00:00<00:00, 2917.70it/s]


In [144]:
a

,all_caps_count,Cont_Punc_Count,Last_Token_Punc,HashTag_Count
0,1,0,0,0
1,5,1,0,0
2,5,0,0,0
3,1,0,0,0
4,2,0,0,0
...,...,...,...,...
95,2,0,0,0
96,1,0,1,1
97,2,0,0,0
98,3,0,0,0


In [84]:
df_training.tweet[1]

'@USER @USER Go home you’re drunk!!! @USER #MAGA #Trump2020 👊🇺🇸👊 URL'